# Construct Model
## 层和块

要想直观地了解块是如何工作的，最简单的方法就是自己实现一个。在实现我们自定义块之前，我们简要总
 结一下每个块必须提供的基本功能:
1. 将输入数据作为其前向传播函数的参数。
2. 通过前向传播函数来生成输出。请注意，输出的形状可能与输入的形状不同。例如，我们上面模型中的 第一个全连接的层接收一个20维的输入，但是返回一个维度为256的输出。
3. 计算其输出关于输入的梯度，可通过其反向传播函数进行访问。通常这是自动发生的。
4. 存储和访问前向传播计算所需的参数。
5. 根据需要初始化模型参数。

- nn.Module

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

In [2]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)
        
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

In [3]:
X = torch.rand(2, 20)

In [4]:
net = MLP()
net(X)

tensor([[-4.7751e-02,  3.0519e-02, -1.3022e-01, -1.4182e-01, -5.0505e-02,
          7.9276e-02, -2.1164e-01, -2.2582e-01,  1.6151e-01, -1.4080e-02],
        [ 1.3360e-01,  1.2580e-04, -2.8988e-01, -1.8293e-01, -4.2609e-02,
         -2.6659e-02, -1.7400e-01, -1.8822e-01, -1.8279e-02, -1.0633e-01]],
       grad_fn=<AddmmBackward>)

- Sequential

In [5]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            self._modules[block] = block
    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

In [6]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.0806, -0.1091, -0.0911,  0.1421, -0.0329, -0.0186,  0.1114,  0.1374,
          0.0353,  0.0677],
        [ 0.0509, -0.0193, -0.2884,  0.1464, -0.0095, -0.0149,  0.2861,  0.2378,
         -0.0259,  0.1294]], grad_fn=<AddmmBackward>)

- customed Sequential

In [7]:
class FixedHiddenMLP(nn.Module): 
    def __init__(self):
        super().__init__()
    # 不计算梯度的随机权重参数。因此其在训练期间保持不变 
        self.rand_weight = torch.rand((20, 20), requires_grad=False) 
        self.linear = nn.Linear(20, 20)
   
    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1) # 复用全连接层。这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
             X /= 2 
        return X.sum()

In [8]:
net = FixedHiddenMLP()
net(X)

tensor(-0.2334, grad_fn=<SumBackward0>)

In [9]:
class NestMLP(nn.Module): 
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)
    def forward(self, X):
        return self.linear(self.net(X))
    
chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.0734, grad_fn=<SumBackward0>)

## 参数管理
- 参数访问

In [10]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[0.5147],
        [0.5059]], grad_fn=<AddmmBackward>)

In [11]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[ 0.2107,  0.1969,  0.1138, -0.3384,  0.2079, -0.0931,  0.2871, -0.1316]])), ('bias', tensor([0.2190]))])


- 目标参数

In [12]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.2190], requires_grad=True)
tensor([0.2190])


In [13]:
net[2].weight.grad == None

True

In [14]:
print(*[(name, param.shape) 
           for name, param in net[0].named_parameters()]) 
print(*[(name, param.shape) 
        for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


- 从嵌套块收集参数

In [15]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())
def block2():
    net = nn.Sequential() 
    for i in range(4):
        # 在这里嵌套
        net.add_module(f'block {i}', block1()) 
    return net
rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[0.3360],
        [0.3360]], grad_fn=<AddmmBackward>)

In [16]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [17]:
rgnet[0][1][0].bias.data

tensor([ 0.0846,  0.3983, -0.1351, -0.3034, -0.2293, -0.4161,  0.1897,  0.3767])

- 初始化参数

In [18]:
def init_normal(m):
    if type(m) == nn.Linear:
            nn.init.normal_(m.weight, mean=0, std=0.01)
            nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0007, -0.0071, -0.0118,  0.0043]), tensor(0.))

In [19]:
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight) 

def init_42(m):
    if type(m) == nn.Linear: 
        nn.init.constant_(m.weight, 42)

In [20]:
net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.3271, -0.2146, -0.2330,  0.5427])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


- customed init

In [21]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
            for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5
net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-0.0000,  0.0000, -0.0000, -9.5366],
        [ 7.5645,  0.0000, -0.0000,  0.0000]], grad_fn=<SliceBackward>)

- 直接设置参数

In [22]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000,  1.0000,  1.0000, -8.5366])

- 参数绑定

In [23]:
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0]) 
net[2].weight.data[0, 0] = 100
# 确保它们实际上是同一个对象，而不只是有相同的值 
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


## 自定义层

In [24]:
import torch
import torch.nn.functional as F 
from torch import nn

class CenteredLayer(nn.Module): 
    def __init__(self):
        super().__init__()
    def forward(self, X): 
        return X - X.mean()

In [25]:
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [26]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data 
        return F.relu(linear)

In [27]:
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[-0.6748, -1.0818,  1.4796],
        [-1.4607, -0.3612, -1.0764],
        [ 0.3376, -2.5370, -0.7482],
        [-0.6351,  0.6045,  1.3811],
        [ 0.1430, -0.0157,  0.7986]], requires_grad=True)

## 读写文件

In [28]:
import torch
from torch import nn
from torch.nn import functional as F

x = torch.arange(4)
torch.save(x, 'x-file')